In [18]:
import pandas as pd
from scipy.stats import mode
import re
import numpy as np
from fuzzywuzzy import process

In [4]:
df_robin = pd.read_csv("data/robin_portmann_stage2.csv")
df_oli = pd.read_csv("data/oliver_heisel_stage2.csv")

In [5]:
df_robin.head()

,brand,name,price,delivery_information,extracted_rating,extracted_gb,phone_color,date,delivery_time_days
0,ASUS,ROG Phone 6,599.00,morgen geliefert,4.4,512.0,Phantom Black,2023-11-24,1
1,ASUS,ROG Phone 6 Diablo Immortal Edition,913.08,Zwischen Mi. 29.11. und Fr. 1.12. geliefert,3.5,512.0,Diablo Immortal Edition,2023-11-24,5
2,ASUS,Zenfone 10,649.00,morgen geliefert,3.6,256.0,Midnight Black,2023-11-24,1
3,ASUS,ROG Phone 7,1040.00,morgen geliefert,4.5,512.0,Storm White,2023-11-24,1
4,ASUS,Zenfone 10,828.00,morgen geliefert,3.9,512.0,Midnight Black,2023-11-24,1


In [6]:
df_oli.head()

,scrapeday,brand,model,storage,color,refurbished,rating,deliverydays,price_CHF
0,bf,XIAOMI,Redmi Note 12,128.0,Onyx Gray,False,4.1250,0,192.09
1,bf,XIAOMI,12 T 5G,256.0,Black,False,4.5263,1,346.54
2,bf,XIAOMI,Redmi 12,128.0,Midnight Black,False,4.4872,1,153.48
3,bf,XIAOMI,Redmi A2,32.0,Black,False,4.0000,1,75.28
4,bf,XIAOMI,Redmi A2,32.0,Light Green,False,4.0000,1,75.28


### first we need to match the date and scrapeday column

In [7]:
df_robin["date"].value_counts()

date
2023-11-24    1955
2023-11-27    1785
2023-12-04    1783
Name: count, dtype: int64

In [8]:
df_oli["scrapeday"].value_counts()

scrapeday
af    2254
bf    2031
cm    1994
Name: count, dtype: int64

In [9]:
#replace the date column values
date_replacements = {
    '2023-11-24': 'bf',
    '2023-11-27': 'cm',
    '2023-12-04': 'af'
}

df_robin["scrapeday"] = df_robin["date"].replace(date_replacements)

In [10]:
df_robin["scrapeday"].value_counts()

scrapeday
bf    1955
cm    1785
af    1783
Name: count, dtype: int64

In [11]:
df_robin["name"].unique()

array(['ROG Phone 6', 'ROG Phone 6 Diablo Immortal Edition', 'Zenfone 10',
       ..., '8i', '9 Pro+', 'GT Neo 3'], dtype=object)

In [12]:
df_robin.head()

,brand,name,price,delivery_information,extracted_rating,extracted_gb,phone_color,date,delivery_time_days,scrapeday
0,ASUS,ROG Phone 6,599.00,morgen geliefert,4.4,512.0,Phantom Black,2023-11-24,1,bf
1,ASUS,ROG Phone 6 Diablo Immortal Edition,913.08,Zwischen Mi. 29.11. und Fr. 1.12. geliefert,3.5,512.0,Diablo Immortal Edition,2023-11-24,5,bf
2,ASUS,Zenfone 10,649.00,morgen geliefert,3.6,256.0,Midnight Black,2023-11-24,1,bf
3,ASUS,ROG Phone 7,1040.00,morgen geliefert,4.5,512.0,Storm White,2023-11-24,1,bf
4,ASUS,Zenfone 10,828.00,morgen geliefert,3.9,512.0,Midnight Black,2023-11-24,1,bf


In [13]:
df_oli.head()

,scrapeday,brand,model,storage,color,refurbished,rating,deliverydays,price_CHF
0,bf,XIAOMI,Redmi Note 12,128.0,Onyx Gray,False,4.1250,0,192.09
1,bf,XIAOMI,12 T 5G,256.0,Black,False,4.5263,1,346.54
2,bf,XIAOMI,Redmi 12,128.0,Midnight Black,False,4.4872,1,153.48
3,bf,XIAOMI,Redmi A2,32.0,Black,False,4.0000,1,75.28
4,bf,XIAOMI,Redmi A2,32.0,Light Green,False,4.0000,1,75.28


In [14]:
#also convert extracted_gb in df_robin to int
df_robin["extracted_gb"] = df_robin["extracted_gb"].astype(int)

In [15]:
#remove all the refurbished phones from olis' dataset
df_oli = df_oli[df_oli["refurbished"] == 0].copy()

df_oli.head()

,scrapeday,brand,model,storage,color,refurbished,rating,deliverydays,price_CHF
0,bf,XIAOMI,Redmi Note 12,128.0,Onyx Gray,False,4.1250,0,192.09
1,bf,XIAOMI,12 T 5G,256.0,Black,False,4.5263,1,346.54
2,bf,XIAOMI,Redmi 12,128.0,Midnight Black,False,4.4872,1,153.48
3,bf,XIAOMI,Redmi A2,32.0,Black,False,4.0000,1,75.28
4,bf,XIAOMI,Redmi A2,32.0,Light Green,False,4.0000,1,75.28


# merging the data

In [16]:
pd.merge(df_robin, df_oli, left_on=["brand", "name", "phone_color", "extracted_gb", "scrapeday"],
         right_on=["brand", "model", "color", "storage", "scrapeday"],
         how="inner", suffixes=("_digitec", "_mediamarkt"))

,brand,name,price,delivery_information,extracted_rating,extracted_gb,phone_color,date,delivery_time_days,scrapeday,model,storage,color,refurbished,rating,deliverydays,price_CHF
0,ASUS,ROG Phone 7,1040.00,morgen geliefert,4.5,512,Storm White,2023-11-24,1,bf,ROG Phone 7,512.0,Storm White,False,5.0000,3,1157.39
1,ASUS,ROG Phone 7 Ultimate,1349.00,morgen geliefert,3.9,512,Storm White,2023-11-24,1,bf,ROG Phone 7 Ultimate,512.0,Storm White,False,4.5238,3,1350.45
2,ASUS,ROG Phone 7,959.00,Zwischen Sa. 25.11. und Do. 30.11. geliefert,4.5,512,Phantom Black,2023-11-24,1,bf,ROG Phone 7,512.0,Phantom Black,False,4.6400,5,1156.43
3,ASUS,ROG Phone 7,878.00,morgen geliefert,3.4,256,Phantom Black,2023-11-24,1,bf,ROG Phone 7,256.0,Phantom Black,False,4.5000,5,1051.00
4,ASUS,ROG Phone 7,940.00,morgen geliefert,3.4,256,Storm White,2023-11-24,1,bf,ROG Phone 7,256.0,Storm White,False,4.5000,5,974.31
5,ASUS,Zenfone 9,593.00,morgen geliefert,4.6,128,Midnight Black,2023-11-24,1,bf,Zenfone 9,128.0,Midnight Black,False,4.7143,1,602.32
6,ASUS,Zenfone 9,746.00,morgen geliefert,4.6,256,Midnight Black,2023-11-24,1,bf,Zenfone 9,256.0,Midnight Black,False,4.4000,7,1453.37
7,ASUS,Zenfone 9,537.33,Zwischen Do. 30.11. und Mo. 4.12. geliefert,4.6,128,Moonlight White,2023-11-24,6,bf,Zenfone 9,128.0,Moonlight White,False,5.0000,1,546.48
8,ASUS,ROG Phone 7 Ultimate,1299.00,morgen geliefert,3.9,512,Storm White,2023-11-27,1,cm,ROG Phone 7 Ultimate,512.0,Storm White,False,4.5238,2,1348.78
9,ASUS,ROG Phone 7,1041.00,morgen geliefert,4.5,512,Storm White,2023-11-27,1,cm,ROG Phone 7,512.0,Storm White,False,5.0000,2,1155.96


Unfortunately, the data is not matching well, this is most likely due to different names of the coloring of the phones. We will therefore first group the data by brand, name and storage and take the average price

In [20]:
#helper function to aggregate by the mode
# def mode_agg(series):
#     return mode(series).mode[0]

def mode_agg(series):
    mode = series.mode()
    if len(mode) > 0:
        return mode.iloc[0]
    else:
        return np.nan
    
df_robin_grouped = df_robin.groupby(["brand", "name", "extracted_gb", "scrapeday"], as_index=False).\
    agg({"price": "mean", "extracted_rating": mode_agg, "delivery_time_days": mode_agg})

In [21]:
df_robin_grouped.head()

,brand,name,extracted_gb,scrapeday,price,extracted_rating,delivery_time_days
0,ASUS,ROG Phone 5 (12GB),256,af,782.005,4.6,1
1,ASUS,ROG Phone 5 (12GB),256,bf,782.005,4.6,1
2,ASUS,ROG Phone 5 (12GB),256,cm,782.005,4.6,1
3,ASUS,ROG Phone 5 (16GB),256,af,804.000,4.7,1
4,ASUS,ROG Phone 5 (16GB),256,bf,804.000,4.7,1


In [22]:
#aggregatig oli's data as well
df_oli_grouped = df_oli.groupby(["brand", "model", "storage", "scrapeday"], as_index=False).\
    agg({"price_CHF": "mean", "rating": "mean", "deliverydays": "median"})

In [23]:
df_oli_grouped.head()

,brand,model,storage,scrapeday,price_CHF,rating,deliverydays
0,ALCATEL,1B 2022 5031G,32.0,af,92.70,5.0,1.0
1,ALCATEL,1B 2022 5031G,32.0,bf,94.64,5.0,1.0
2,ALCATEL,1B 2022 5031G,32.0,cm,94.52,5.0,1.0
3,ALCATEL,5033FR,16.0,af,64.57,NaN,3.0
4,ALCATEL,5033FR,16.0,bf,72.30,NaN,5.0


In [24]:
df_robin_grouped.query("brand == 'Apple'")

,brand,name,extracted_gb,scrapeday,price,extracted_rating,delivery_time_days
105,Apple,iPhone 11,64,af,443.500000,4.7,1
106,Apple,iPhone 11,64,bf,459.000000,4.7,1
107,Apple,iPhone 11,64,cm,443.500000,4.7,1
108,Apple,iPhone 11,128,af,455.666667,4.7,1
109,Apple,iPhone 11,128,bf,499.000000,4.7,1
...,...,...,...,...,...,...,...
355,Apple,iPhone 14 Plus,256,bf,869.000000,4.6,1
356,Apple,iPhone 14 Plus,256,cm,869.000000,4.7,1
357,Apple,iPhone 14 Plus,512,af,900.000000,4.7,5
358,Apple,iPhone 14 Plus,512,bf,993.000000,4.6,7


In [25]:
df_oli_grouped.query("brand == 'Apple'")

,brand,model,storage,scrapeday,price_CHF,rating,deliverydays


In [26]:
df_oli_grouped["brand"].unique()

array(['ALCATEL', 'ALLVIEW', 'APPLE', 'ASUS', 'BEAFON', 'BLACKBERRY',
       'BLACKVIEW', 'CARBON', 'CAT', 'CROSSCALL', 'CUBOT', 'DOOGEE',
       'DORO', 'EMPORIA', 'ENERGIZER', 'FAIRPHONE', 'FOSSIBOT', 'FUNKER',
       'FYSIC', 'GIGASET', 'GOOGLE', 'HAMMER', 'HONOR', 'HUAWEI',
       'IIIF150', 'INFINIX', 'LENOVO', 'LEXIBOOK', 'LG', 'MAXCOM',
       'MOTOROLA', 'MP', 'MYPHONE', 'Myphone', 'NOKIA', 'NOTHING',
       'OLYMPIA', 'OTROS', 'OUKITEL', 'PEAQ', 'POCO', 'POCOPHONE', 'QUBO',
       'REALME', 'REPHONE', 'RUGGEAR', 'SAMSUNG', 'SONY', 'SPC',
       'SWISSVOICE', 'TCL', 'TECNO', 'TELECOM', 'ULEFONE', 'UMIDIGI',
       'VIVANCO', 'WIKO', 'XIAOMI', 'ZTE'], dtype=object)

In [27]:
df_robin_grouped["brand"].unique()

array(['ASUS', 'Acer', 'Alcatel', 'Allview', 'Apple', 'Bea-Fon',
       'Blabloo', 'BlackBerry', 'Blackview', 'Brondi', 'Cat', 'Crosscall',
       'Cubot', 'Custom', 'Cyrus', 'Denso', 'Doogee', 'Doro', 'Emporia',
       'Energizer', 'Fairphone', 'Gigaset', 'Google', 'HTC', 'Hammer',
       'Honor', 'Huawei', 'I.safe Mobile', 'IIIf150', 'Inapa', 'Infinix',
       'Inoi', 'Kazam', 'Krüger&Matz', 'LG', 'Maxcom', 'Microsoft',
       'Motorola', 'Myphone', 'NGM', 'NoName', 'Nokia', 'Nothing',
       'Nubia', 'OPPO', 'Olympia', 'OnePlus', 'Ordissimo', 'Oukitel',
       'POCO', 'Rephone', 'Rug Gear', 'Samsung', 'Sony', 'Syco', 'TCL',
       'TE Connectivity', 'TIM', 'Tecno', 'Telekom', 'Ulefone', 'Umi',
       'Umidigi', 'Vivo', 'Vodafone', 'Volla', 'Xiaomi', 'ZTE', 'iGet',
       'realme'], dtype=object)

the brand names in oli's df are all upper case whereas most of the brand names in robin's df have only the first capital letter, we will therefore make them also all uppercase

In [28]:
#convert to uppercase
df_robin_grouped["brand"] = df_robin_grouped["brand"].str.upper()

In [29]:
#now we can merge the data
df_merged = pd.merge(df_robin_grouped, df_oli_grouped,
                     left_on=["brand", "name", "extracted_gb","scrapeday"],
                     right_on=["brand", "model", "storage", "scrapeday"],
                     how="inner", suffixes=("_digitec", "_mediamarkt"))

df_merged.head()

,brand,name,extracted_gb,scrapeday,price,extracted_rating,delivery_time_days,model,storage,price_CHF,rating,deliverydays
0,ASUS,ROG Phone 7,256,af,921.0,3.4,1,ROG Phone 7,256.0,1015.500,4.5000,1.0
1,ASUS,ROG Phone 7,256,bf,909.0,3.4,1,ROG Phone 7,256.0,1012.655,4.5000,5.0
2,ASUS,ROG Phone 7,256,cm,999.0,3.4,1,ROG Phone 7,256.0,1048.460,4.5000,4.0
3,ASUS,ROG Phone 7,512,af,1079.0,4.5,1,ROG Phone 7,512.0,1133.650,4.8269,2.0
4,ASUS,ROG Phone 7,512,bf,999.5,4.5,1,ROG Phone 7,512.0,1156.910,4.8200,4.0


In [30]:
df_merged["name"].unique()

array(['ROG Phone 7', 'ROG Phone 7 Ultimate', 'Zenfone 9', 'iPhone 11',
       'iPhone 12', 'iPhone 12 mini', 'iPhone 13', 'iPhone 13 Pro',
       'iPhone 13 mini', 'iPhone 14', 'iPhone 14 Plus', 'iPhone 14 Pro',
       'iPhone 14 Pro Max', 'iPhone 15', 'iPhone 15 Pro', 'iPhone SE',
       'BL5000', 'BL8800 Pro', 'BV4900', 'BV4900 Pro', 'BV6300 Pro',
       'BV8800', 'S42 H+', 'S62 Pro', 'S75', '8200', 'SMART.5 mini', '5',
       'GS4', 'GS5 LITE', 'Pixel 2', 'Pixel 6', 'Pixel 6 Pro', 'Pixel 6a',
       'Pixel 7', 'Pixel 7 Pro', 'Pixel 7a', 'Pixel 8', 'Pixel 8 Pro',
       'Iron 4', '50', '70', '90', '90 Lite', 'Magic 4 Lite 5G',
       'Magic5 Pro', 'X8', 'Mate 50 Pro', 'Mate X3', 'Nova 10',
       'Nova 10 SE', 'P30', 'P30 Pro', 'P40 Lite', 'P40 Lite 5G',
       'P40 Pro', 'P50 Pro', 'P60 Pro', 'Y5 (2019)', 'Defy',
       'Edge 30 Neo', 'Edge 40 Pro', 'Moto E22', 'Moto E6i', 'Moto G22',
       'Moto G31', 'Moto G32', 'Moto G52', 'Moto G72', 'Moto G9 Play',
       'Razr 40', 'C12', 'C

In [31]:
df_robin_grouped["name"].unique()

array(['ROG Phone 5 (12GB)', 'ROG Phone 5 (16GB)', 'ROG Phone 5s (12GB)',
       ..., 'Narzo 50A', 'Narzo 50A Prime',
       'Narzo 50A Prime 64GB Flash Black [16.7cm (6.6 ") IPS LCD screen, Android 11, 50MP triple camera'],
      dtype=object)

In [32]:
df_oli_grouped["model"].unique()

array(['1B 2022 5031G', '5033FR', 'PIXI 4-5 (3G) 5010D BLACK', ...,
       'P606F01-GREY', 'V40 Pro', 'V40 Vita'], dtype=object)

Looking at the phone names, sometimes the storage and/or the color is also in there, which makes matching the phones even more difficult, we will try to remove them as good as possible by using regex

In [33]:
# Function to clean phone names
def clean_phone_name(name):
    # Remove storage capacity patterns like '64GB'
    name = re.sub(r'\d+GB', '', name)
    # Remove common color names and other patterns
    name = re.sub(r'\b(schwarz|grün|black|white|blue|red|gold|silver)\b', '', name, flags=re.IGNORECASE)
    # Remove additional unwanted patterns (e.g., '(NEU)', '-')
    name = re.sub(r'\(.*?\)|-', '', name)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# Apply the cleaning function
df_robin_grouped['cleaned_phone_name'] = df_robin_grouped['name'].apply(clean_phone_name)
df_oli_grouped['cleaned_phone_name'] = df_oli_grouped['model'].apply(clean_phone_name)

In [34]:
df_merged = pd.merge(df_robin_grouped, df_oli_grouped,
                     left_on=["brand", "cleaned_phone_name", "extracted_gb","scrapeday"],
                     right_on=["brand", "cleaned_phone_name", "storage", "scrapeday"],
                     how="inner", suffixes=("_digitec", "_mediamarkt"))
df_merged.head()

,brand,name,extracted_gb,scrapeday,price,extracted_rating,delivery_time_days,cleaned_phone_name,model,storage,price_CHF,rating,deliverydays
0,ASUS,ROG Phone 7,256,af,921.0,3.4,1,ROG Phone 7,ROG Phone 7,256.0,1015.500,4.5000,1.0
1,ASUS,ROG Phone 7,256,bf,909.0,3.4,1,ROG Phone 7,ROG Phone 7,256.0,1012.655,4.5000,5.0
2,ASUS,ROG Phone 7,256,cm,999.0,3.4,1,ROG Phone 7,ROG Phone 7,256.0,1048.460,4.5000,4.0
3,ASUS,ROG Phone 7,512,af,1079.0,4.5,1,ROG Phone 7,ROG Phone 7,512.0,1133.650,4.8269,2.0
4,ASUS,ROG Phone 7,512,bf,999.5,4.5,1,ROG Phone 7,ROG Phone 7,512.0,1156.910,4.8200,4.0


we now get more matching phones, by removing the storage/color of the phones in the names. At this point one would have to manually remove them further

Additionally, we can also try to match the names using fuzzy-matching

In [35]:
"""def get_best_match(row, matcher_column, choices, limit=1):
    best_matches = process.extract(row[matcher_column], choices, limit=limit)
    return best_matches[0][0] if best_matches else None

# Applying fuzzy matching
df_robin_grouped.apply(get_best_match, axis=1, matcher_column='name', choices=df_oli_grouped['model'].tolist())"""

"def get_best_match(row, matcher_column, choices, limit=1):\n    best_matches = process.extract(row[matcher_column], choices, limit=limit)\n    return best_matches[0][0] if best_matches else None\n\n# Applying fuzzy matching\ndf_robin_grouped.apply(get_best_match, axis=1, matcher_column='name', choices=df_oli_grouped['model'].tolist())"

# Adjusting the column names

In [36]:
df_merged = df_merged.rename(columns={
    'price': 'price_digitec_CHF',
    'price_CHF': 'price_mediamarkt_CHF',
    'extracted_rating': 'rating_digitec',
    'rating': 'rating_mediamarkt',
    'delivery_time_days': 'delivery_time_days_digitec',
    'deliverydays': 'delivery_time_days_mediamarkt',
    'extracted_gb': 'phone_storage'})

In [37]:
df_merged.to_csv("data/joined_data.csv", index=False)